In [5]:
import pandas as pd

df = pd.read_csv('~/Desktop/NHL-Salary-Predictions/data/final_df.csv')

# df = df.sort_values('2020-21', ascending=False)[['fullName', 'name', '2020-21']][:10]
for col in df.columns[df.columns.str.contains('Salary_')]:
    df[col] = df[col].fillna(0.0).apply(lambda x: "${:,.2f}".format(x))

df

,Unnamed: 0,fullName,id,jerseyNumber,code,name,type,abbreviation,Team_Number,birthDate,...,Salary_2017-18,Salary_2018-19,Salary_2019-20,Salary_2020-21,Salary_2013-14,Salary_2011-12,Salary_2012-13,Salary_2010-11,Salary_2008-09,Salary_2009-10
0,0,Josh Jacobs,8477972,40.0,D,Defenseman,Defenseman,D,1,1996-02-15,...,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00
1,1,A.J. Greer,8478421,42.0,L,Left Wing,Forward,LW,1,1996-12-14,...,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00
2,2,P.K. Subban,8474056,76.0,D,Defenseman,Defenseman,D,1,1989-05-13,...,"$11,000,000.00","$10,000,000.00","$10,000,000.00","$8,000,000.00",$0.00,$0.00,$0.00,$0.00,$0.00,$0.00
3,3,Scott Wedgewood,8475809,41.0,G,Goalie,Goalie,G,1,1992-08-14,...,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00
4,4,Matt Tennyson,8476807,7.0,D,Defenseman,Defenseman,D,1,1990-04-23,...,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
940,957,Dylan Coghlan,8479639,52.0,D,Defenseman,Defenseman,D,54,1998-02-19,...,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00
941,958,Nolan Patrick,8479974,19.0,C,Center,Forward,C,54,1998-09-19,...,$0.00,$0.00,$0.00,"$874,125.00",$0.00,$0.00,$0.00,$0.00,$0.00,$0.00
942,960,Nicolas Hague,8479980,14.0,D,Defenseman,Defenseman,D,54,1998-12-05,...,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00
943,961,Logan Thompson,8480313,36.0,G,Goalie,Goalie,G,54,1997-02-25,...,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00


In [6]:
[col for col in df.columns]

['Unnamed: 0',
 'fullName',
 'id',
 'jerseyNumber',
 'code',
 'name',
 'type',
 'abbreviation',
 'Team_Number',
 'birthDate',
 'currentAge',
 'birthCity',
 'birthStateProvince',
 'birthCountry',
 'nationality',
 'height',
 'weight',
 'active',
 'alternateCaptain',
 'captain',
 'rookie',
 'shootsCatches',
 'rosterStatus',
 'season20',
 'timeOnIce20',
 'assists20',
 'goals20',
 'pim20',
 'shots20',
 'games20',
 'hits20',
 'powerPlayGoals20',
 'powerPlayPoints20',
 'powerPlayTimeOnIce20',
 'evenTimeOnIce20',
 'penaltyMinutes20',
 'faceOffPct20',
 'shotPct20',
 'gameWinningGoals20',
 'overTimeGoals20',
 'shortHandedGoals20',
 'shortHandedPoints20',
 'shortHandedTimeOnIce20',
 'blocked20',
 'plusMinus20',
 'points20',
 'shifts20',
 'timeOnIcePerGame20',
 'evenTimeOnIcePerGame20',
 'shortHandedTimeOnIcePerGame20',
 'powerPlayTimeOnIcePerGame20',
 'ot20',
 'shutouts20',
 'ties20',
 'wins20',
 'losses20',
 'saves20',
 'powerPlaySaves20',
 'shortHandedSaves20',
 'evenSaves20',
 'shortHandedShot

In [4]:
df = df[df['name'] != 'Goalie']
df

,fullName,id,jerseyNumber,code,name,type,abbreviation,Team_Number,birthDate,currentAge,...,Salary_2017-18,Salary_2018-19,Salary_2019-20,Salary_2020-21,Salary_2013-14,Salary_2011-12,Salary_2012-13,Salary_2010-11,Salary_2008-09,Salary_2009-10
0,Josh Jacobs,8477972,40.0,D,Defenseman,Defenseman,D,1,1996-02-15,25,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,A.J. Greer,8478421,42.0,L,Left Wing,Forward,LW,1,1996-12-14,24,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,P.K. Subban,8474056,76.0,D,Defenseman,Defenseman,D,1,1989-05-13,32,...,11000000.0,10000000.0,10000000.0,8000000.0,0.0,0.0,0.0,0.0,NaN,NaN
4,Matt Tennyson,8476807,7.0,D,Defenseman,Defenseman,D,1,1990-04-23,31,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,Ryan Murray,8476850,22.0,D,Defenseman,Defenseman,D,1,1993-09-27,27,...,0.0,0.0,4500000.0,4700000.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
939,Brett Howden,8479353,21.0,C,Center,Forward,C,54,1998-03-29,23,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN
940,Dylan Coghlan,8479639,52.0,D,Defenseman,Defenseman,D,54,1998-02-19,23,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
941,Nolan Patrick,8479974,19.0,C,Center,Forward,C,54,1998-09-19,22,...,0.0,0.0,0.0,874125.0,0.0,0.0,0.0,0.0,NaN,NaN
942,Nicolas Hague,8479980,14.0,D,Defenseman,Defenseman,D,54,1998-12-05,22,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN


In [89]:
df = pd.read_csv('~/Desktop/NHL-Salary-Predictions/data/final_df.csv').drop('Unnamed: 0', axis=1)

df_name_pos_sal = df[['fullName', 'Salary_2020-21']] \
                        .rename(columns={'fullName':'Player Name', "Salary_2020-21":'Salary 2020-21'}) \
                        .sort_values('Salary 2020-21', ascending=False)

df_name_pos_sal['Rank'] = range(1, len(df_name_pos_sal) + 1)

df_name_pos_sal = df_name_pos_sal[['Rank', 'Player Name', 'Salary 2020-21']]

for col in df_name_pos_sal.columns[df_name_pos_sal.columns.str.contains('Salary')]:
    df_name_pos_sal[col] = df_name_pos_sal[col].fillna(0.0).apply(lambda x: "${:,.2f}".format(x))

In [3]:
from dash import html, dash_table, Input, Output, callback
import dash_bootstrap_components as dbc
import pandas as pd

df = pd.read_csv('~/Desktop/NHL-Salary-Predictions/data/final_df.csv').drop('Unnamed: 0', axis=1)

df_name_pos_sal = df[['fullName', 'Salary_2020-21']] \
                        .rename(columns={'fullName':'Player Name', "Salary_2020-21":'Salary 2020-21'}) \
                        .sort_values('Salary 2020-21', ascending=False)

df_name_pos_sal['Rank'] = range(1, len(df_name_pos_sal) + 1)

df_name_pos_sal = df_name_pos_sal[['Rank', 'Player Name', 'Salary 2020-21']]

for col in df_name_pos_sal.columns[df_name_pos_sal.columns.str.contains('Salary')]:
    df_name_pos_sal[col] = df_name_pos_sal[col].fillna(0.0) # .apply(lambda x: "${:,.2f}".format(x))

layout = html.Div(
    className="row",
    children=[
        html.Div([
            html.H3('Player Salaries', style={'text-align': 'center', 'text-decoration': 'underline'}),
            dash_table.DataTable(columns =[{"name": i, "id": i} for i in df_name_pos_sal.columns],
                                 data=df_name_pos_sal.to_dict('records'),
                                 # page_size=page_size,
                                 filter_action='native',
                                 sort_action="native",
                                 style_data={'color': 'black', 'backgroundColor': 'white', 'border': '1px solid black'},
                                 style_header={'backgroundColor': 'white', 'color': 'black', 'fontWeight': 'bold', 'border': '1px solid black'},
                                 id='player_tbl')],
            style={'height': 450, 'width':450, 'overflowX': 'scroll', 'display': 'inline-block', 'vertical-align': 'top', 'margin-left': '3vw', 'margin-top': '3vw'}
            )
    ]
)

@callback(Output('tbl_out', 'childeren'), Input('player_tbl', 'active_cell'))
def update_graphs(active_cell):
    return str(active_cell) if active_cell else 'Click the table'

In [94]:
money

In [92]:
df_name_pos_sal['Salary 2020-21'].

301    $15,910,000.00
298    $15,000,000.00
79     $13,000,000.00
638    $13,000,000.00
345    $12,000,000.00
            ...      
883             $0.00
897             $0.00
899             $0.00
902             $0.00
917             $0.00
Name: Salary 2020-21, Length: 945, dtype: object

In [75]:
df['Salary_2019-20'].sort_values(ascending=False)

320    $969,896.00
268    $925,000.00
820    $925,000.00
819    $925,000.00
25     $925,000.00
          ...     
391          $0.00
392          $0.00
394          $0.00
395          $0.00
944          $0.00
Name: Salary_2019-20, Length: 945, dtype: object

In [62]:
from dash import Dash, Input, Output, callback, dash_table
import pandas as pd
import dash_bootstrap_components as dbc

df = pd.read_csv('https://git.io/Juf1t')

app = Dash(external_stylesheets=[dbc.themes.BOOTSTRAP])

app.layout = dbc.Container([
    dbc.Label('Click a cell in the table:'),
    dash_table.DataTable(df.to_dict('records'),[{"name": i, "id": i} for i in df.columns], id='tbl'),
    dbc.Alert(id='tbl_out'),
])

@callback(Output('tbl_out', 'children'), Input('tbl', 'active_cell'))
def update_graphs(active_cell):
    return str(active_cell) if active_cell else "Click the table"

In [64]:
df.to_dict('records')

[{'State': 'California',
  'Number of Solar Plants': 289,
  'Installed Capacity (MW)': 4395,
  'Average MW Per Plant': 15.3,
  'Generation (GWh)': 10826},
 {'State': 'Arizona',
  'Number of Solar Plants': 48,
  'Installed Capacity (MW)': 1078,
  'Average MW Per Plant': 22.5,
  'Generation (GWh)': 2550},
 {'State': 'Nevada',
  'Number of Solar Plants': 11,
  'Installed Capacity (MW)': 238,
  'Average MW Per Plant': 21.6,
  'Generation (GWh)': 557},
 {'State': 'New Mexico',
  'Number of Solar Plants': 33,
  'Installed Capacity (MW)': 261,
  'Average MW Per Plant': 7.9,
  'Generation (GWh)': 590},
 {'State': 'Colorado',
  'Number of Solar Plants': 20,
  'Installed Capacity (MW)': 118,
  'Average MW Per Plant': 5.9,
  'Generation (GWh)': 235},
 {'State': 'Texas',
  'Number of Solar Plants': 12,
  'Installed Capacity (MW)': 187,
  'Average MW Per Plant': 15.6,
  'Generation (GWh)': 354},
 {'State': 'North Carolina',
  'Number of Solar Plants': 148,
  'Installed Capacity (MW)': 669,
  'Avera

In [61]:
df['fullName'].to_list()

['Josh Jacobs',
 'A.J. Greer',
 'P.K. Subban',
 'Scott Wedgewood',
 'Matt Tennyson',
 'Ryan Murray',
 'Damon Severson',
 'Connor Carrick',
 'Aaron Dell',
 'Andreas Johnsson',
 'Will Butcher',
 'Miles Wood',
 'Ryan Graves',
 'Jonas Siegenthaler',
 'Pavel Zacha',
 'Mackenzie Blackwood',
 'Nicholas Merkley',
 'Jesper Bratt',
 'Nathan Bastian',
 'Michael McLeod',
 'Janne Kuokkanen',
 'Nico Hischier',
 'Jesper Boqvist',
 'Ty Smith',
 'Yegor Sharangovich',
 'Jack Hughes',
 'Anders Lee',
 'Johnny Boychuk',
 'Braydon Coburn',
 'Travis Zajac',
 'Cory Schneider',
 'Andy Greene',
 'Leo Komarov',
 'Cal Clutterbuck',
 'Semyon Varlamov',
 'Thomas Hickey',
 'Josh Bailey',
 'Jordan Eberle',
 'Matt Martin',
 'Kyle Palmieri',
 'Richard Panik',
 'Casey Cizikas',
 'Brock Nelson',
 'Jean-Gabriel Pageau',
 'Scott Mayfield',
 'Adam Pelech',
 'Ryan Pulock',
 'Ross Johnston',
 'Michael Dal Colle',
 'Ilya Sorokin',
 'Cole Bardreau',
 'Mathew Barzal',
 'Anthony Beauvillier',
 'Austin Czarnik',
 'Dmytro Timashov'

In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10 entries, 308 to 62
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   fullName  10 non-null     object 
 1   name      10 non-null     object 
 2   2020-21   10 non-null     float64
dtypes: float64(1), object(2)
memory usage: 320.0+ bytes


<class 'pandas.core.frame.DataFrame'>
Int64Index: 10 entries, 308 to 62
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   fullName  10 non-null     object
 1   name      10 non-null     object
 2   2020-21   10 non-null     object
dtypes: object(3)
memory usage: 320.0+ bytes


In [15]:
df[df['fullName'] == 'Alex Ovechkin']

,fullName,id,jerseyNumber,code,name,type,abbreviation,Team_Number,birthDate,currentAge,...,2017-18,2018-19,2019-20,2020-21,2013-14,2011-12,2012-13,2010-11,2008-09,2009-10
418,Alex Ovechkin,8471214,8.0,L,Left Wing,Forward,LW,15,1985-09-17,35,...,10000000.0,10000000.0,10000000.0,10000000.0,9000000.0,9000000.0,5268293.0,9000000.0,9000000.0,9000000.0


In [3]:
from jupyter_dash import JupyterDash

import dash
from dash import dcc
from dash import html
import pandas as pd

import plotly.express as px

app = JupyterDash(__name__)

# -------------------------------------------------------------------------------------------------------

df = pd.read_csv('~/Desktop/NHL-Salary-Predictions/data/cleaned_players_df.csv')

# -------------------------------------------------------------------------------------------------------


app = JupyterDash(__name__)

# assume you have a "long-form" data frame
# see https://plotly.com/python/px-arguments/ for more options
fig = px.bar(df, x="Fruit", y="Amount", color="City", barmode="group")

app.layout = html.Div(children=[
    html.H1(children='Hello Dash'),

    html.Div(children='''
        Dash: A web application framework for your data.
    '''),

    dcc.Graph(
        id='example-graph',
        figure=fig
    )
])


app.run_server(debug=True, mode='inline' )

In [2]:
# Run this app with `python app.py` and
# visit http://127.0.0.1:8050/ in your web browser.
from jupyter_dash import JupyterDash

import dash
from dash import dcc
from dash import html
import pandas as pd

import plotly.express as px

app = JupyterDash(__name__)

colors = {
    'background': '#111111',
    'text': '#7FDBFF'
}

# assume you have a "long-form" data frame
# see https://plotly.com/python/px-arguments/ for more options
df = pd.DataFrame({
    "Fruit": ["Apples", "Oranges", "Bananas", "Apples", "Oranges", "Bananas"],
    "Amount": [4, 1, 2, 2, 4, 5],
    "City": ["SF", "SF", "SF", "Montreal", "Montreal", "Montreal"]
})

fig = px.bar(df, x="Fruit", y="Amount", color="City", barmode="group")

fig.update_layout(
    plot_bgcolor=colors['background'],
    paper_bgcolor=colors['background'],
    font_color=colors['text']
)

app.layout = html.Div(style={'backgroundColor': colors['background']}, children=[
    html.H1(
        children='Hello Dash',
        style={
            'textAlign': 'center',
            'color': colors['text']
        }
    ),

    html.Div(children='Dash: A web application framework for your data.', style={
        'textAlign': 'center',
        'color': colors['text']
    }),

    dcc.Graph(
        id='example-graph-2',
        figure=fig
    )
])


app.run_server(debug=True, mode='inline')

In [1]:
from dash import Dash, html, dcc
from jupyter_dash import JupyterDash


app = JupyterDash(__name__)

app.layout = html.Div([
    html.Div(children=[
        html.Label('Dropdown'),
        dcc.Dropdown(['New York City', 'Montréal', 'San Francisco'], 'Montréal'),

        html.Br(),
        html.Label('Multi-Select Dropdown'),
        dcc.Dropdown(['New York City', 'Montréal', 'San Francisco'],
                     ['Montréal', 'San Francisco'],
                     multi=True),

        html.Br(),
        html.Label('Radio Items'),
        dcc.RadioItems(['New York City', 'Montréal', 'San Francisco'], 'Montréal'),
    ], style={'padding': 10, 'flex': 1}),

    html.Div(children=[
        html.Label('Checkboxes'),
        dcc.Checklist(['New York City', 'Montréal', 'San Francisco'],
                      ['Montréal', 'San Francisco']
        ),

        html.Br(),
        html.Label('Text Input'),
        dcc.Input(value='MTL', type='text'),

        html.Br(),
        html.Label('Slider'),
        dcc.Slider(
            min=0,
            max=9,
            marks={i: f'Label {i}' if i == 1 else str(i) for i in range(1, 6)},
            value=5,
        ),
    ], style={'padding': 10, 'flex': 1})
], style={'display': 'flex', 'flex-direction': 'row'})

app.run_server(debug=True, mode='inline')

In [2]:
import pandas as pd

df = pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/master/gapminder2007.csv')
# add an id column and set it as the index
# in this case the unique ID is just the country name, so we could have just
# renamed 'country' to 'id' (but given it the display name 'country'), but
# here it's duplicated just to show the more general pattern.
df['id'] = df['country']
df.set_index('id', inplace=True, drop=False)

In [3]:
df

,country,pop,continent,lifeExp,gdpPercap,id
id,,,,,,
Afghanistan,Afghanistan,31889923.0,Asia,43.828,974.580338,Afghanistan
Albania,Albania,3600523.0,Europe,76.423,5937.029526,Albania
Algeria,Algeria,33333216.0,Africa,72.301,6223.367465,Algeria
Angola,Angola,12420476.0,Africa,42.731,4797.231267,Angola
Argentina,Argentina,40301927.0,Americas,75.320,12779.379640,Argentina
...,...,...,...,...,...,...
Vietnam,Vietnam,85262356.0,Asia,74.249,2441.576404,Vietnam
West Bank and Gaza,West Bank and Gaza,4018332.0,Asia,73.422,3025.349798,West Bank and Gaza
"Yemen, Rep.","Yemen, Rep.",22211743.0,Asia,62.698,2280.769906,"Yemen, Rep."


In [4]:
df.loc['Afghanistan']

country      Afghanistan
pop           31889923.0
continent           Asia
lifeExp           43.828
gdpPercap     974.580338
id           Afghanistan
Name: Afghanistan, dtype: object